<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" /> 


&nbsp;
# Administración activa Vs Administración pasiva del capital
## Equipo B

***Emilio de la Torre***


***José Raúl Gallardo***


***Adrian Garcia*** 


La idea es replicar un ETF con posiciones en directo. Te han asignado la tarea de hacer un ejercicio simple de replicar la construcción de un portafolio de inversión y un criterio de rebalanceo lo más parecido a un ETF de ishares que tu selecciones. Esto con la finalidad de comparar los resultados de haber rebalanceado periódicamente el portafolio, ó, no haberlo hecho y continuar todo un año con las mismas posiciones.

## Limpieza y extracción de datos

### Importando librerías a utilizar

In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import plotly.graph_objs as go #Esta librería necesita ser instalada desde terminal 'pip install plotly'

### Importando los datos del ETF

Función para convertir datos string a float.

In [95]:
def s2f(to_convert):
    f = float(to_convert.replace(',',''))
    return f

Función para extraer información de los etfs (csv) y los tickers de cada uno.

In [96]:
def get_data(nFiles, file_title):
    df = []
    dr = np.arange(1,nFiles+1)
    df = [pd.read_csv(file_title+str(dr[i])+'.csv', header=1, index_col=None, parse_dates=False, skip_blank_lines=True, skiprows=9) for i in range(len(dr))]
    data = [df[i][(df[i]['Asset Class'] == 'Equity')] for i in range(len(df))] #filtramos solo con equity (acciones)
    tickers = [list(data[i].iloc[:,0]) for i in range(len(data))] #extracción de tickers de cada mes
    return data, tickers

In [97]:
data_etf, tickers = get_data(13, 'IVE_Holdings')

Función para sacar todos los tickers (valores únicos) de todos los archivos juntos.

In [98]:
#Función depende de la anterior
def unique_tickers(tickers): 
    nFiles = len(tickers) #archivos que tenemos
    ticker = [] 
    for i in range(nFiles):
        ticker = ticker + tickers[i] #Anexando los tickers de cada archivo en una sola lista
    df = pd.DataFrame(ticker) #Se convierte en dataframe para poder sacar valores únicos
    unique_tickers = df.iloc[:,0].unique() #.unique() regresa los valores únicos
    return unique_tickers #Todos los tickers que se van a necesitar de quandl

In [99]:
all_tickers = unique_tickers(tickers)
len(all_tickers)

453

Función para extraer el número de acciones en circulación de los etf's de ishares.

In [100]:
def n_shares(nFiles, file_title):
    dr = np.arange(1,nFiles+1)
    df = [pd.read_csv(file_title+str(dr[i])+'.csv', index_col=None, parse_dates=False, skip_blank_lines=True, nrows=6) for i in range(len(dr))]
    all_shares = [df[i].iloc[3,0] for i in range(len(df))]
    all_shares_int = [int(float(all_shares[i].replace(',',''))) for i in range(len(all_shares))]
    return all_shares_int

In [101]:
etf_shares = n_shares(13, 'IVE_Holdings')

### Extracción de precios de Quandl

In [102]:
import quandl
quandl.ApiConfig.api_key = "JyBxQPPNx_LTemwn-VbD" # Api para acceso a Quandl

Obteniendo acceso a los precios de Quandl/WikiPrices.

In [103]:
d_quandl = quandl.get_table('WIKI/PRICES', ticker = [all_tickers], qopts = {'columns': ['ticker', 'date', 'close']},
                        date = {'gte': '2017-03-01', 'lte': '2018-04-01'}, paginate=True)
d_quandl.head()

,ticker,date,close
None,,,
0,ZION,2018-03-27,52.020000
1,ZION,2018-03-26,53.240000
2,ZION,2018-03-23,51.230000
3,ZION,2018-03-22,53.070000
4,ZION,2018-03-21,55.100000


A continuación se ordena el dataframe para tener los precios de los tickers en columnas

In [104]:
stock_prices = d_quandl.set_index('date')
stock_prices = stock_prices.pivot(columns='ticker') #Pivotear el df para tener los tickers como columnas
stock_prices.head()

close                                                      \
ticker             A       AAL        AAP       ABC       ABT        ACN   
date                                                                       
2017-03-01 52.040000 47.090000 156.680000 92.230000 45.590000 124.070000   
2017-03-02 51.380000 45.720000 157.360000 91.940000 45.690000 124.350000   
2017-03-03 51.980000 46.820000 155.350000 92.230000 45.250000 123.640000   
2017-03-06 51.250000 45.310000 154.250000 89.990000 45.130000 123.240000   
2017-03-07 51.210000 44.840000 151.990000 88.480000 45.000000 124.140000   

                                                       ...               \
ticker           ADM        ADS      ADSK       AEE    ...           XL   
date                                                   ...                
2017-03-01 46.510000 247.620000 88.930000 54.760000    ...    41.120000   
2017-03-02 45.480000 243.940000 87.320000 55.120000    ...    40.610000   
2017-03-03 45.200000 244.630000 85.320000 54.890000    ...    40.360000   
2017-03-06 44.760000 245.450000 82.940000 54.890000    ...    39.900000   
2017-03-07 44.460000 245.050000 82.070000 54.820000    ...    40.130000   

                                                                       \
ticker          XLNX       XOM      XRAY      XRX       XYL      YHOO   
date                                                                    
2017-03-01 59.680000 83.020000 64.500000 7.390000 49.550000 46.240000   
2017-03-02 59.350000 83.300000 63.650000 7.290000 48.890000 45.940000   
2017-03-03 58.740000 82.460000 63.360000 7.360000 48.730000 45.910000   
2017-03-06 58.810000 82.830000 62.750000 7.450000 48.530000 45.640000   
2017-03-07 58.750000 82.520000 62.550000 7.360000 48.780000 45.730000   

                                           
ticker           YUM        ZBH      ZION  
date                                       
2017-03-01 65.570000 118.890000 46.530000  
2017-03-02 65.210000 118.020000 44.960000  
2017-03-03 65.030000 118.280000 45.360000  
2017-03-06 64.500000 117.220000 45.330000  
2017-03-07 64.020000 117.070000 45.280000  

[5 rows x 450 columns]

Se elimina el multindex para poder manupular los datos mejor y se llenan los NaN con 0.

In [105]:
stock_prices.columns = stock_prices.columns.droplevel(0) #Se elimina el header principal para evitar el multindex
stock_prices = stock_prices.fillna(0) #NaN se rellenan con 0
stock_prices.head()

ticker,A,AAL,AAP,ABC,ABT,ACN,ADM,ADS,ADSK,AEE,...,XL,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION
date,,,,,,,,,,,,,,,,,,,,,
2017-03-01,52.040000,47.090000,156.680000,92.230000,45.590000,124.070000,46.510000,247.620000,88.930000,54.760000,...,41.120000,59.680000,83.020000,64.500000,7.390000,49.550000,46.240000,65.570000,118.890000,46.530000
2017-03-02,51.380000,45.720000,157.360000,91.940000,45.690000,124.350000,45.480000,243.940000,87.320000,55.120000,...,40.610000,59.350000,83.300000,63.650000,7.290000,48.890000,45.940000,65.210000,118.020000,44.960000
2017-03-03,51.980000,46.820000,155.350000,92.230000,45.250000,123.640000,45.200000,244.630000,85.320000,54.890000,...,40.360000,58.740000,82.460000,63.360000,7.360000,48.730000,45.910000,65.030000,118.280000,45.360000
2017-03-06,51.250000,45.310000,154.250000,89.990000,45.130000,123.240000,44.760000,245.450000,82.940000,54.890000,...,39.900000,58.810000,82.830000,62.750000,7.450000,48.530000,45.640000,64.500000,117.220000,45.330000
2017-03-07,51.210000,44.840000,151.990000,88.480000,45.000000,124.140000,44.460000,245.050000,82.070000,54.820000,...,40.130000,58.750000,82.520000,62.550000,7.360000,48.780000,45.730000,64.020000,117.070000,45.280000


Para tener los pesos de las acciones presentes en Quandl es necesario hacer match entre los tickers del etf y los de la fuente de precios.

In [106]:
tickers_quandl = [stock_prices.columns[i] for i in range(len(stock_prices.columns))] #Extraemos los tickers disponibles de quandl

In [107]:
df_w = pd.DataFrame(tickers_quandl, columns=['Ticker']) # Tickers se hacen un dataframe
# Se unen todos los datos de los dataframes con los tickers originales
all_df_etf = [df_w.set_index('Ticker').join(data_etf[i].set_index('Ticker'), on='Ticker') for i in range(len(data_etf))]
# Limpiando los dataframes para eliminar activos que no fueron necesarios en cada mes
final_etf_data = [all_df_etf[i].dropna() for i in range(len(all_df_etf))]

## ¿Cómo se calcula el NAV (Net Asset Value) de un ETF?

Se suma el valor total de cada uno de los instrumentos que componen el ETF, y se divide entre el numero de certificados que el ETF emitió. 

$$ NAV = \frac{\sum (Títulos\,de\,cada\,acción * precio)}{Acciones\,en\,circulación\,del\,etf}$$

### Cálculo del NAV 

#### NAV y rendimientos mensuales de los archivos de iShares

Se calculan los rendimientos mensuales del etf, tomando en cuenta el market value $(acciones*precio)$.

In [108]:
nav_mensual = [sum([s2f(final_etf_data[i].iloc[j,5]) for j in range(len(final_etf_data[i]))])/etf_shares[i] 
                      for i in range(len(final_etf_data))] #Se calcula el nav de los archivos originales

rend_etf_mensual = [((nav_mensual[i]/nav_mensual[i-1])-1) for i in (np.arange(1,len(nav_mensual)))] #Rendimiento mensual

In [109]:
fig = go.FigureWidget() #Se crea la figura 
fig.add_scatter(x=['Mar-17','Abr-17','May-17','Jun-17','Jul-17','Ago-17','Sept-17','Oct-17','Nov-17',
                              'Dic-17','Ene-18','Feb-18','Mar-18'] ,y=nav_mensual)
fig.layout.title = 'NAV mensual IVE Etf'
fig.update_layout(xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text="Meses")), #Títulos a los ejes
                 yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text="Precios")))
fig.show()

In [110]:
fig = go.FigureWidget()
fig.add_scatter(x=['Abr-17','May-17','Jun-17','Jul-17','Ago-17','Sept-17','Oct-17','Nov-17',
                              'Dic-17','Ene-18','Feb-18','Mar-18'] ,y=rend_etf_mensual)
fig.layout.title = 'Rendimiento mensual IVE Etf'
fig.update_layout(xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text="Meses")),
                 yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text="Rendimiento en decimal")))
fig.show()

#### Cálculo del NAV diario con distintos escenarios

Se analizan diferentes escenarios del etf:

 * No hay rebalanceos, el etf inicial es el que tendremos hasta fin de año, sólo se ve afectado por los precios.
 * Sí hay rebalanceos, se parte desde el mes inicial y posteriormente se hace un rebalanceo por año; así con los 12 meses:
     * Combinaciones: [1,2], [1,3], [1,4]...

Fechas a tomar en cuenta:

In [111]:
dates = ['2017-04','2017-05','2017-06','2017-07','2017-08','2017-09','2017-10','2017-11','2017-12','2018-01',
         '2018-02','2018-03','2018-04'] #Fechas de interés para el NAV diario de quandl

Función para calcular el rendimiento mensual y la desviación estándar del etf por mes.

In [112]:
# La variable tickers_Nmes debe ser ingresada como una lista.
# La variable meses debe ser ingresada como una lista anidada, la principal deberá ser de la longitud de la variable tickers_Nmes
def nav_quandl(tickers_Nmes, meses):
    monthly_shares_df = [] #Acciones mensuales de cada mes
    nav_diario = [] #Nav diaro por mes
    desv_est = [] #Desv std mensual
    rend_mensual = [] #Rend mensual
    etf_data = [] #Etf de ishares para extraer los tickers
    shares_out = [] #acciones en circulación de cada activo
    for i in (tickers_Nmes):
        shares = [s2f(final_etf_data[i].iloc[j,4]) for j in range(len(final_etf_data[i].iloc[:,4]))]
        shares_df = pd.DataFrame(shares,columns=['shares'])
        monthly_shares_df.append(shares_df)
        etf_data.append(final_etf_data[i])
        shares_out.append(etf_shares[i])
    for i in range(len(tickers_Nmes)):
        for j in range(len(meses[i])-1):
            nav_diario = np.dot(stock_prices.loc[:,etf_data[i].index.values] #Se filtra df por mes de interés y se calcula el NAV
                                [np.logical_and(stock_prices.index >= np.datetime64(meses[i][j]), #multiplicacion vectorial
                                                stock_prices.index < np.datetime64(meses[i][j+1]))],monthly_shares_df[i])/shares_out[i]
            desv_est.append(np.std(nav_diario))
            rend_mensual.append(float(((nav_diario[-1]/nav_diario[0])-1))) #rend mensual último dato del mes y primero
    return rend_mensual, desv_est

Obteniendo las combinaciones posibles.

In [113]:
rend = []
desv_std = []
for i in range(2,len(dates)): #Escenario 2-12
    x,y = nav_quandl([1,i],[dates[:i],dates[i-1::]])
    rend.append(x)
    desv_std.append(y)

r1, s1 = nav_quandl([1],[dates]) #Se calcula primer escenario, sin rebalanceo
rend.insert(0,r1)
desv_std.insert(0,s1)

Lo que nos interesa del siguiente bloque es calcular la desviación estándar/riesgo del etf original.

In [114]:
ignore, desv_original_etf = nav_quandl(list(np.arange(1,13)),[['2017-04','2017-05'],['2017-05','2017-06'],['2017-06','2017-07'],
                                                          ['2017-07','2017-08'],['2017-08','2017-09'],['2017-09','2017-10'],
                                                          ['2017-10','2017-11'],['2017-11','2017-12'],['2017-12','2018-01'],
                                                          ['2018-01','2018-02'],['2018-02','2018-03'],['2018-03','2018-04']])
# Desv estandar de cada etf original iShares

### Rebalanceo propuesto

Se cuenta con un presupuesto de 1'000,000.00 USD para comprar acciones y armar el rebalanceo del ETF. EL criterio consiste en lo siguiente:
* Escoger inicialmente el top 5 de acciones con mayor rendimiento por mes.
    * Las proporciones del dinero a invertir se repartirán de la siguiente manera: 30% el activo con mayor rendimiento, 25% el segundo, 20% el tercero, 15% el cuarto y 10% el último.

Presupuesto para acciones:

In [115]:
cap = 1000000
money = [cap*.30, cap*.25, cap*.20, cap*.15, cap*.10] #proporciones de capital

In [116]:
dates2 = ['2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09','2017-10','2017-11','2017-12','2018-01',
         '2018-02','2018-03']
shares_out_n = 1000000 #acciones en circulacion del etf propuesto

In [117]:
n_shares= []
for i in range(len(dates2)-1):
    prices = stock_prices[np.logical_and(stock_prices.index >= np.datetime64(dates2[i]),stock_prices.index < 
                                         np.datetime64(dates2[i+1]))]
    m_rend = [((prices.iloc[-1,:]/prices.iloc[0,:])-1).nlargest()][0] #nlargest extrae el top 5 de rendimientos
    n_shares.append(m_rend)

In [118]:
shares_per_c  = []
for i in range(len(n_shares)):
    shares_etf_n = np.ceil(money/stock_prices.loc[:,n_shares[i].index.values] #Se calculan las acciones a comprar por ticker
                           [np.logical_and(stock_prices.index >= np.datetime64(dates2[i]),
                                           stock_prices.index < np.datetime64(dates2[i+1]))].iloc[0,:]) #Valor al inicio de c/mes
    for j in range(len(shares_etf_n)):
        if shares_etf_n[j] == np.inf:
            shares_etf_n[j] = 1000 #Reemplazamos infinitos por 1000 numero estandar
    shares_per_c.append(shares_etf_n)

In [119]:
desv_est_n = []
rend_mensual_n = []
dates_n_etef = [['2017-04','2017-05'],['2017-05','2017-06'],['2017-06','2017-07'],
                                                          ['2017-07','2017-08'],['2017-08','2017-09'],['2017-09','2017-10'],
                                                          ['2017-10','2017-11'],['2017-11','2017-12'],['2017-12','2018-01'],
                                                          ['2018-01','2018-02'],['2018-02','2018-03'],['2018-03','2018-04']]
for i in range(len(shares_per_c)):
    for j in range(len(dates_n_etef[i])-1):
                nav_etf_p = np.dot(stock_prices.loc[:,shares_per_c[i].index.values]
                                    [np.logical_and(stock_prices.index >= np.datetime64(dates_n_etef[i][j]),
                                                    stock_prices.index < np.datetime64(dates_n_etef[i][j+1]))],
                                    shares_per_c[i])/shares_out_n
                desv_est_n.append(np.std(nav_etf_p))
                rend_mensual_n.append(float(((nav_etf_p[-1]/nav_etf_p[0])-1)))

## Medidas del desempeño

### Comparación de rendimientos

In [120]:
r_dates = dates[:-1]
colmns = ['Caso '+str(i) for i in range(1,len(rend)+1)] #Generacion de columnas df
colmns.insert(0, "Original iShares")
rend_table = pd.DataFrame(index=r_dates, columns=colmns)
rend_table['Original iShares'] = rend_etf_mensual
for i in colmns[1::]:
    case = colmns[1::].index(i) #Se extrae el numero de la columna para buscarlo en los rendimientos mensuales de quandl
    rend_table[i] = rend[case] #Columna se le asigna el caso correspondiente
pd.set_option('precision', 4)
rend_table['Proposal'] = rend_mensual_n
rend_table

,Original iShares,Caso 1,Caso 2,Caso 3,Caso 4,Caso 5,Caso 6,Caso 7,Caso 8,Caso 9,Caso 10,Caso 11,Caso 12,Proposal
2017-04,-0.001850,0.000572,0.000572,0.000572,0.000572,0.000572,0.000572,0.000572,0.000572,0.000572,0.000572,0.000572,0.000572,0.047096
2017-05,-0.017715,-0.006936,-0.007641,-0.006936,-0.006936,-0.006936,-0.006936,-0.006936,-0.006936,-0.006936,-0.006936,-0.006936,-0.006936,-0.007952
2017-06,0.029238,0.002359,0.002378,0.009276,0.002359,0.002359,0.002359,0.002359,0.002359,0.002359,0.002359,0.002359,0.002359,-0.049223
2017-07,0.012229,0.001410,0.001383,0.001350,0.004816,0.001410,0.001410,0.001410,0.001410,0.001410,0.001410,0.001410,0.001410,0.057951
2017-08,-0.014716,-0.021721,-0.021724,-0.021774,-0.021724,-0.017452,-0.021721,-0.021721,-0.021721,-0.021721,-0.021721,-0.021721,-0.021721,0.000705
2017-09,0.027067,0.026588,0.026597,0.026372,0.026419,0.026372,0.026232,0.026588,0.026588,0.026588,0.026588,0.026588,0.026588,0.019081
2017-10,0.010590,0.002691,0.002712,0.002949,0.002940,0.003025,0.003419,0.004659,0.002691,0.002691,0.002691,0.002691,0.002691,0.003039
2017-11,0.032502,0.023639,0.023592,0.023571,0.023655,0.023547,0.022639,0.022537,0.022432,0.023639,0.023639,0.023639,0.023639,0.003087
2017-12,0.016736,0.011326,0.011298,0.011323,0.011312,0.011428,0.011109,0.011193,0.011221,0.015239,0.011326,0.011326,0.011326,0.009929
2018-01,0.039998,0.035522,0.035485,0.035258,0.035387,0.035287,0.035498,0.035521,0.035519,0.031262,0.031276,0.035522,0.035522,0.016275


### Comparación de volatilidad

In [121]:
r_dates = dates[:-1]
colmns = ['Caso '+str(i) for i in range(1,len(rend)+1)]
colmns.insert(0, "Original iShares")
desv_table = pd.DataFrame(index=r_dates, columns=colmns)
desv_table['Original iShares'] = desv_original_etf
for i in colmns[1::]:
    case = colmns[1::].index(i) #Se extrae el numero de la columna para buscarlo en los rendimientos mensuales de quandl
    desv_table[i] = desv_std[case] #Columna se le asigna el caso correspondiente
pd.set_option('precision', 4)
desv_table['Proposal'] = desv_est_n
desv_table

,Original iShares,Caso 1,Caso 2,Caso 3,Caso 4,Caso 5,Caso 6,Caso 7,Caso 8,Caso 9,Caso 10,Caso 11,Caso 12,Proposal
2017-04,21.685445,21.685445,21.685445,21.685445,21.685445,21.685445,21.685445,21.685445,21.685445,21.685445,21.685445,21.685445,21.685445,0.262218
2017-05,0.648849,0.628055,0.648849,0.628055,0.628055,0.628055,0.628055,0.628055,0.628055,0.628055,0.628055,0.628055,0.628055,0.015826
2017-06,0.573561,0.578208,0.571615,0.573561,0.578208,0.578208,0.578208,0.578208,0.578208,0.578208,0.578208,0.578208,0.578208,0.021125
2017-07,0.466445,0.459677,0.454686,0.461206,0.466445,0.459677,0.459677,0.459677,0.459677,0.459677,0.459677,0.459677,0.459677,0.028800
2017-08,0.940602,0.995355,0.984848,1.000717,1.003503,0.940602,0.995355,0.995355,0.995355,0.995355,0.995355,0.995355,0.995355,0.007554
2017-09,1.138475,1.103494,1.092121,1.101508,1.105368,1.107672,1.138475,1.103494,1.103494,1.103494,1.103494,1.103494,1.103494,0.006585
2017-10,0.368382,0.366058,0.362257,0.366824,0.366563,0.367374,0.372179,0.368382,0.366058,0.366058,0.366058,0.366058,0.366058,0.012407
2017-11,0.954586,0.942103,0.931886,0.942519,0.943578,0.943855,0.953532,0.953373,0.954586,0.942103,0.942103,0.942103,0.942103,0.004027
2017-12,0.790828,0.669874,0.661114,0.671639,0.673983,0.676507,0.676422,0.681203,0.684373,0.790828,0.669874,0.669874,0.669874,0.019811
2018-01,1.591860,1.686322,1.667662,1.689202,1.698290,1.699415,1.716301,1.720099,1.723843,1.589028,1.591860,1.686322,1.686322,0.014302


### Comparación Sharpe Ratio

Extrayendo tasa libre de riesgo de Estados Unidos.

In [122]:
treasury = 'Treasury Yield.xlsx' # Archivo de tasas
start_date = '03-01-2017'
end_date = '04-01-2018'
t_yield = pd.read_excel(treasury, header = 0) #se lee archivo de tasas
t_yield['Date'] = pd.to_datetime(t_yield['Date']) #Fechas se convierten a datetime para filtar
t_y = t_yield[(t_yield['Date'] > start_date) & (t_yield['Date'] <= end_date)] #Se filtra dataframe por fecha
t_y = t_y.sort_values(by=['Date']) #Se ordena en base a fechas
rf_rates = [t_y[(t_y['Date'] > dates[i]) & (t_y['Date'] < dates[i+1])].iloc[-1,1] for i in range(len(dates)-1)] #Se extrae el último dato de cada mes

In [123]:
sharpe = rend_table.copy()
for i in range(len(sharpe.iloc[:,0])):
    for j in range(len(sharpe.iloc[0,:])):
        sharpe.iat[i,j] = (rend_table.iat[i,j]-(rf_rates[i]/12/100))/desv_table.iat[i,j] #se saca el sharpe por coordenada
pd.set_option('display.float_format', lambda x: '%.6f' % x)      #formato de 6 decimales
sharpe

,Original iShares,Caso 1,Caso 2,Caso 3,Caso 4,Caso 5,Caso 6,Caso 7,Caso 8,Caso 9,Caso 10,Caso 11,Caso 12,Proposal
2017-04,-0.000111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.177444
2017-05,-0.028407,-0.012184,-0.012881,-0.012184,-0.012184,-0.012184,-0.012184,-0.012184,-0.012184,-0.012184,-0.012184,-0.012184,-0.012184,-0.547726
2017-06,0.049756,0.002870,0.002935,0.014952,0.002870,0.002870,0.002870,0.002870,0.002870,0.002870,0.002870,0.002870,0.002870,-2.363187
2017-07,0.024431,0.001255,0.001208,0.001119,0.008538,0.001255,0.001255,0.001255,0.001255,0.001255,0.001255,0.001255,0.001255,1.983271
2017-08,-0.016487,-0.022617,-0.022862,-0.022549,-0.022437,-0.019396,-0.022617,-0.022617,-0.022617,-0.022617,-0.022617,-0.022617,-0.022617,-0.011522
2017-09,0.023072,0.023369,0.023621,0.023215,0.023177,0.023086,0.022339,0.023369,0.023369,0.023369,0.023369,0.023369,0.023369,2.776211
2017-10,0.026509,0.005096,0.005208,0.005789,0.005771,0.005990,0.006969,0.010408,0.005096,0.005096,0.005096,0.005096,0.005096,0.178461
2017-11,0.033053,0.024083,0.024297,0.024001,0.024063,0.023942,0.022746,0.022643,0.022504,0.024083,0.024083,0.024083,0.024083,0.530819
2017-12,0.019814,0.015315,0.015476,0.015271,0.015201,0.015316,0.014846,0.014865,0.014837,0.017921,0.015315,0.015315,0.015315,0.447361
2018-01,0.024378,0.020358,0.020564,0.020167,0.020135,0.020063,0.019989,0.019958,0.019913,0.018924,0.018899,0.020358,0.020358,1.054689


## Conclusiones

A lo largo del laboratorio nos encontramos con distintos problemas para trabajar los archivos debido a que contenían mucha información y debiamos analizar todo, el hecho de que usaramos compresión de listas nos permitio eficientar el tiempo en el que se ejecutaban las operaciones.

Con medidas descriptivas tales como rendimiento, volatilidad y Sharpe evaluamos 12 escenarios diferentes del ETF que contemplaban rebalanceo en diferentes periodos de tiempo; el interés sobre esto fue para generar nosotros un criterio de rebalanceo para nuestra replica del ETF con posiciones en directo y fue comparado con los escenarios previamente calculados.

Con esta herramienta tan poderosa como es el Python, pudimos replicar un ETF con nuestro propio criterio de rebalanceo sin necesidad de contratar un gestor, con esto nos ahorramos comisiones.
Al crear muchas alternativas o caminos diferentes podemos darnos cuenta de cual sería el más cercano al rendimiento óptimo y así decidir si nos quedamos pasivamente con el ETF o si mejor le dedicamos tiempo y le damos una gestión activa, esto dependerá de los beneficios extra que nos traiga modificar nuestras posiciones con el ETF.

## Notas

> Para los cálculos solo se están usando los activos de "Equity".

> En total tenemos 453 activos diferentes en el etf, quandl sólo permitió extraer 449; algunos en algún periodo de tiempo ya tenían NaN, se rellenaron con 0.

> Las ponderaciones mensuales no van a dar 100%, debido a la ausencia de algunos activos.